## Continued from NLP_Sentiment_TDIF
## Calculates similarity matrix from song to song


In [40]:
# import things
import pandas as pd
import numpy as np 
from scipy import spatial
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
# Import BeautifulSoup into your workspace
from bs4 import BeautifulSoup  
import re
import string
import operator
from __future__ import division
import datetime
#import nltk
#nltk.download()  # Download text data sets, including stop words
from nltk.corpus import stopwords # Import the stop word list
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [41]:
def cos_dis(your_song_bow_vec, other_song_bow_vec):
    result = spatial.distance.cosine(your_song_bow_vec, other_song_bow_vec)
    return result

In [42]:
lyrics_df = pd.read_pickle('LyricsDF_with_BoW&tdif_sparse')

CosDis2 = pd.read_pickle('CosDis_tfidf_reduced')
CosDis2 = CosDis2.as_matrix()

song_i_like = 'Bohemian Rhapsody'

In [43]:
#get date of inputted song
input_date=lyrics_df[lyrics_df['track_name']==song_i_like].date.values[0]
input_date = input_date.astype('datetime64[D]')

#find songs with similar sentiment
sentiment = lyrics_df[lyrics_df['track_name']==song_i_like].compound_sentiment.values[0]
lyrics_df['dist1'] = lyrics_df.compound_sentiment.map((lambda x: abs(sentiment - x)))	

#define similarity as diff between your song's lexical diversity & that of other songs
lex = lyrics_df[lyrics_df['track_name']==song_i_like].lex_diversity.values[0]
lyrics_df['dist2'] = lyrics_df.lex_diversity.map((lambda x: abs(lex - x)))


In [46]:
##HOLY CRAP putting in the command below makes the following for loop run 
## SO MUCH FASTER.  But why???
pd.options.mode.chained_assignment = None  # default='warn'

lyrics_recent = lyrics_df[lyrics_df['date']>datetime.date(2013,1,1)]
lyrics_recent = lyrics_recent.reset_index()
del lyrics_recent['index']

index1 = lyrics_df[lyrics_df['track_name']==song_i_like].index

lyrics_recent['dist3'] = 2.0
num_recent_songs = len(lyrics_recent)
for j in range(num_recent_songs):
    lyrics_recent['dist3'][j] = CosDis2[index1, j]

lyrics_recent['dist_tot'] = lyrics_recent['dist1'] + lyrics_recent['dist2'] + lyrics_recent['dist3']
 

In [47]:
song1 = 'Bohemian Rhapsody'
song2 = 'Cool for the Summer'

bow1 = lyrics_df[lyrics_df['track_name']==song1].tfidf_array.values[0].toarray()
bow2 = lyrics_df[lyrics_df['track_name']==song2].tfidf_array.values[0].toarray()
print 'size bag of words vectors = ', np.shape(bow1), ' and ', np.shape(bow2)
cos_distance = cos_dis(bow1,bow2)
print 'cosine distance between ', song1, ' and ', song2 , ' = '
print cos_distance

bow1 = lyrics_df[lyrics_df['track_name']==song1].tfidf_array.values[0].toarray()
bow2 = lyrics_recent[lyrics_recent['track_name']==song2].tfidf_array.values[0].toarray()
print 'size bag of words vectors = ', np.shape(bow1), ' and ', np.shape(bow2)
cos_distance = cos_dis(bow1,bow2)
print 'cosine distance between ', song1, ' and ', song2 , ' = '
print cos_distance

index1 = lyrics_df[lyrics_df['track_name']==song1].index
index2 = lyrics_recent[lyrics_recent['track_name']==song2].index

print CosDis2.shape
print 'index1 =', index1, 'index2 =', index2
print CosDis2[index1, index2]

print lyrics_recent[lyrics_recent['track_name']==song2]['dist3']

size bag of words vectors =  (1, 5000)  and  (1, 5000)
cosine distance between  Bohemian Rhapsody  and  Cool for the Summer  = 
0.965182248585
size bag of words vectors =  (1, 5000)  and  (1, 5000)
cosine distance between  Bohemian Rhapsody  and  Cool for the Summer  = 
0.965182248585
(15868, 920)
index1 = Int64Index([5979], dtype='int64') index2 = Int64Index([656], dtype='int64')
[ 0.96518225]
656    0.965182
Name: dist3, dtype: float64


In [48]:
lyrics_recent['dist_tot'] = lyrics_recent['dist1'] + lyrics_recent['dist2'] + lyrics_recent['dist3']
print lyrics_recent.head(5)

   level_0                     hot100 artist          hot100 track  \
0    14708                        AWOLNATION                  Sail   
1    14873                     The Lumineers                Ho Hey   
2    14878  Justin Bieber Featuring Ludacris  All Around The World   
3    14899                    Mumford & Sons           I Will Wait   
4    14911                   Imagine Dragons           Radioactive   

             track_name      artist_name        track_spotify_id  \
0                  Sail       AWOLNATION  4VUGq8KUTVv5YnMqU6nkDa   
1                Ho Hey    The Lumineers  3MhUoQd6LJjITdDty9phGv   
2  All Around the World    Justin Bieber                           
3           I Will Wait   Mumford & Sons  20PpIH3XleVmYMpyY6nGBs   
4           Radioactive  Imagine Dragons  62yJjFtgkhUrXktIoSjgP2   

                                         lyrics_body       date  \
0  SAIL!\nThis is how I show my love.\nI made it ... 2013-06-22   
1  Ho! Hey! Ho! Hey!\n(Ho!) I've bee

In [50]:
nreturn = 10
result = lyrics_recent.sort_values('dist_tot')[:nreturn]
print result

     level_0                                      hot100 artist  \
162    15108                        2 Chainz Featuring Pharrell   
544    15490                  Hoodie Allen Featuring Ed Sheeran   
889    15837                                              Drake   
691    15637                                         Luke Bryan   
57     15003                                    Imagine Dragons   
599    15545  Kendrick Lamar Featuring Bilal, Anna Wise & Sn...   
515    15461                            Ne-Yo Featuring Juicy J   
552    15498                                     Dierks Bentley   
453    15399                               Eminem Featuring Sia   
56     15002                      A$AP Rocky Featuring Skrillex   

           hot100 track          track_name  \
162       Feds Watching       Feds Watching   
544        All About It        All About It   
889                Hype                Hype   
691       Strip It Down       Strip It Down   
57               Demons    